In [1]:
import pickle
import os

In [2]:
# with open(os.path.join("data","yelp_2017","dict.pkl"),"rb") as f:
#     word_to_ind = pickle.load(f,encoding="bytes")
# with open(os.path.join("data","yelp_2017","word_emb.pkl"),"rb") as f:
#     emb = pickle.load(f,encoding="bytes")

In [3]:
# max(word_to_ind.values()),emb.shape

In [4]:
import sqlite3

conn = sqlite3.connect('yelp_dataset.db')
c = conn.cursor()

def Select(q):
    return c.execute(q)

def Update(q):
    c.execute(q)
    conn.commit()
    
        
# conn.close()

In [5]:
# x=Update("create view RevAB_CA as select \
# User.id as user_id, User.review_count as user_review_count, User.average_rating as user_rating, \
# Business.city as city, Business.state as state, Business.ratings as item_rating, Business.id as item_id, \
# Reviews.id as review_id, Reviews.ts as ts, Reviews.rating as review_rating \
# from Reviews \
# Inner JOIN User on User.id = Reviews.user_id \
# INNER JOIN Business on Business.id = Reviews.business_id \
# where Business.city = \"Las Vegas\"")

In [6]:
data = Select("Select * from RevAB_CA where user_review_count > 15 LIMIT 10000")
#

In [7]:
query = data
data = []
for row in query:
    data += [row]

In [8]:
items = list(set([x[6] for x in data]))
users = list(set([x[0] for x in data]))
# items_ind = {items[i]:i for i in range(len(items))}

# users_ind = {users[i]:i for i in range(len(users))}

In [9]:
len(items),len(users),len(data[0])

(1015, 5661, 11)

In [10]:
from collections import defaultdict 
rev_u = defaultdict(lambda: defaultdict(dict))
rev_i = defaultdict(lambda: defaultdict(dict))
for review in data:
    user = review[0]
    item = review[6]
    text = review[10]#o
    rev_u [user][item] = text
    rev_i [item][user] = text
rev_u = dict(rev_u)
rev_i = dict(rev_i)

In [11]:
import difflib
import numpy as np
filename = 'glove.6B.50d.txt'
def loadGloVe(filename):
    vocab = set()
    embd = {}
    file = open(filename,'r')
    word = None
    with open(filename,encoding="utf-8") as f:
        try:
            for line in f:
                row = line.strip().split(' ')
                word = row[0]
                vocab.add(row[0])
                embd[word] = np.array(row[1:])
            
        except Exception as e:
            print(word)
    print('Loaded GloVe!')
    file.close()
    return vocab,embd
vocab_set,embd = loadGloVe(filename)
# vocab_size = len(vocab)
# vocab_set = set(vocab)
# embedding_dim = len(embd[0])
# embedding = np.asarray(embd)


                    

Loaded GloVe!


In [12]:
max_len = 50

In [91]:
import keras
from keras.models import Sequential
from keras.layers import Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers import Dense,Dropout,BatchNormalization, SimpleRNN, GRU, LSTM,Bidirectional,Conv2D,Merge,Flatten,MaxPooling1D#, FactorizationMachinesLayer 

In [79]:
from keras.layers.core import *
from keras.engine.topology import Layer
from keras import initializers, activations

class FactorizationMachinesLayer(Layer):
    '''Factorization Machines layer.

    # Arguments
        output_dim: int > 0.
        k: k of Factorization Machines
        init: name of initialization function for the weights of the layer
            (see [initializations](../initializations.md)),
            or alternatively, Theano function to use for weights
            initialization. This parameter is only relevant
            if you don't pass a `weights` argument.
        activation: name of activation function to use
            (see [activations](../activations.md)),
            or alternatively, elementwise Theano function.
            If you don't specify anything, no activation is applied
            (ie. "linear" activation: a(x) = x).
        weights: list of Numpy arrays to set as initial weights.
            The list should have 2 elements, of shape `(input_dim, output_dim)`
            and (output_dim,) for weights and biases respectively.
        W_regularizer: instance of [WeightRegularizer](../regularizers.md)
            (eg. L1 or L2 regularization), applied to the main weights matrix.
        b_regularizer: instance of [WeightRegularizer](../regularizers.md),
            applied to the bias.
        activity_regularizer: instance of [ActivityRegularizer](../regularizers.md),
            applied to the network output.
        W_constraint: instance of the [constraints](../constraints.md) module
            (eg. maxnorm, nonneg), applied to the main weights matrix.
        b_constraint: instance of the [constraints](../constraints.md) module,
            applied to the bias.
        bias: whether to include a bias
            (i.e. make the layer affine rather than linear).
        input_dim: dimensionality of the input (integer). This argument
            (or alternatively, the keyword argument `input_shape`)
            is required when using this layer as the first layer in a model.

    # Input shape
        nD tensor with shape: `(nb_samples, ..., input_dim)`.
        The most common situation would be
        a 2D input with shape `(nb_samples, input_dim)`.

    # Output shape
        nD tensor with shape: `(nb_samples, ..., output_dim)`.
        For instance, for a 2D input with shape `(nb_samples, input_dim)`,
        the output would have shape `(nb_samples, output_dim)`.
    '''
    def __init__(self, output_dim, init='glorot_uniform',
                 activation=None, weights=None, k=2,
                 W_regularizer=None, b_regularizer=None, activity_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, input_dim=None, **kwargs):
        self.init = initializers.get(init)
        self.activation = activations.get(activation)
        self.output_dim = output_dim
        self.input_dim = input_dim
        self.k = k

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.initial_weights = weights
#         self.input_spec = [InputSpec(ndim='2+')]
        
        if self.input_dim:
            kwargs['input_shape'] = (self.input_dim,)
        super(FactorizationMachinesLayer, self).__init__(**kwargs)
        print("jnsea")

    def build(self, input_shape):
        assert len(input_shape) >= 2
        print("build")
        input_dim = input_shape[-1]
        self.input_dim = input_dim
        print("input_dim",input_dim)
#         self.input_spec = [InputSpec(dtype=K.floatx(),
#                                      ndim='2+')]
# 
        self.W = self.add_weight((input_dim, self.output_dim),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.V = self.add_weight((self.output_dim, input_dim, self.k),
                                 initializer=self.init,
                                 name='{}_V'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((self.output_dim,),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None
        

        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True
        print("buildN")

    def call(self, x, mask=None):
        output = K.sum(K.square(K.dot(x, self.V)) - K.dot(K.square(x), K.square(self.V)), 2)/2
        output += K.dot(x, self.W)
        if self.bias:
            output += self.b
        return self.activation(output)

    def get_output_shape_for(self, input_shape):
        print(self.input_dim)
        assert input_shape and len(input_shape) >= 2
        assert input_shape[-1] and input_shape[-1] == self.input_dim
        output_shape = list(input_shape)
        output_shape[-1] = self.output_dim
        return tuple(output_shape)

    def get_config(self):
        config = {'output_dim': self.output_dim,
                  'init': self.init.__name__,
                  'activation': self.activation.__name__,
                  'W_regularizer': self.W_regularizer.get_config() if self.W_regularizer else None,
                  'b_regularizer': self.b_regularizer.get_config() if self.b_regularizer else None,
                  'activity_regularizer': self.activity_regularizer.get_config() if self.activity_regularizer else None,
                  'W_constraint': self.W_constraint.get_config() if self.W_constraint else None,
                  'b_constraint': self.b_constraint.get_config() if self.b_constraint else None,
                  'bias': self.bias,
                  'input_dim': self.input_dim}
        base_config = super(FactorizationMachinesLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


# def FM():
# #     model = Sequential()
#     inputs = keras.layers.Input(shape=(100,))
#     fc_ = Dense(100)(inputs)
# #     model.add(Reshape((1,100)))
#     fm = FactorizationMachinesLayer(output_dim=1)(fc_)#,k=8,activation="relu", init="TruncatedNormal"))
# #     model.add(Flatten())
# #     fc = Dense(1)(fc_)
#     return keras.models.Model(inputs=inputs,outputs=fm)
# # fm = FactorizationMachinesLayer(1,k=8)
# # fm.build(input_shape=(1,100))
# # fm.get_output_shape_for((5,100))
# FM()#.summary()


In [137]:
def deepCONN(user_shape,item_shape):
    model1 = Sequential()
    model1.add (Conv2D(32, (3,user_shape[1]),
                 input_shape=user_shape, activation="tanh"))
       
#     model1.add(Flatten())
#     model1.add(Reshape((1,)))
#     model1.add(MaxPooling1D(pool_size=(32)))  
    model1.add(Dense(1,activation="relu"))
    model1.add(Flatten())
    model1.add(Dense(50,activation="relu"))
    model1.add(BatchNormalization())
                     
                     
    model2 = Sequential()
    model2.add (Conv2D(32, (3,item_shape[1]),
                 input_shape=item_shape, activation="tanh"))
#     model2.add(Flatten())
#     model2.add(Reshape((1,)))
#     model2.add(MaxPooling1D(pool_size=(32)))
    model2.add(Dense(1,activation="relu"))
    model2.add(Flatten())
#     model2.Reshape(())
    model2.add(Dense(50,activation="relu"))
    model2.add(BatchNormalization())
                     
    model = Sequential()
    model.add(Merge([model1,model2],mode='concat'))
#     model.add(Dense(32,activation="relu"))
    model.add(Dropout(0.4))
#     model1.add(BatchNormalization())
#     model.add(Reshape((100,1)))
    
#     _model = Sequential()
#     _model.add(Merge([model,model],mode="sum"))
#     model.add(FactorizationMachinesLayer(output_dim=1,k=8,activation="relu", init="TruncatedNormal"))
    model.add(Dense(1,activation="sigmoid"))
    return model,model1,model2

    

In [138]:
model,model1,model2 = deepCONN((max_len,50,1),(max_len,50,1))
model.summary()
model1.summary()
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_16 (Merge)             (None, 100)               0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_83 (Dense)             (None, 1)                 101       
Total params: 15,131
Trainable params: 14,931
Non-trainable params: 200
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_48 (Conv2D)           (None, 48, 1, 32)         4832      
_________________________________________________________________
dense_79 (Dense)             (None, 48, 1, 1)          33        
_________________________________________________________________
flat

c:\python3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [66]:
# import numpy 
# def embedding(words):
#     embed = []
#     for word in words:
#         embed += [embd[word]]
#     for _ in range(100-len(words)):
#         embed += [[0]*50]
#     return numpy.array(embed)

In [67]:
import numpy
def embedding(text,maxlen):
    text = text.split()
    count = 0
    _w = []
    for word in text:
        word = word.lower()
        if word in vocab_set:
            _w += [embd[word]]
        elif word[:-1] in vocab_set and word[-1] == ".":
            _w += [embd[word[:-1]]]
            if len(_w) == max_len:
                break
            _w += [embd[word[-1]]]
        else:
            count += 1
        if len(_w) == max_len:
            break
#     print(len(_w))
    for _ in range(maxlen-len(_w)):
        _w += [numpy.array([0]*50)]
    ar= numpy.array(_w)
    ar = numpy.reshape(ar,(maxlen,50,1))
    return ar

In [55]:
user_emb, item_emb = {},{}
for user in rev_u:
    user_emb[user] = embedding(" ".join([rev_u[user][item] for item in rev_u[user] ]),max_len)
for item in rev_i:
    item_emb[item] = embedding(" ".join([rev_i[item][user] for user in rev_i[item]]),max_len)

In [124]:
# for user in rev_u:
#     user_emb[user] = embedding(user_emb[user])
# for item in rev_i:
#     item_emb[item] = embedding(item_emb[item])

In [126]:
X_user = []
X_item = []
Y = []
for i in range(5000):
    user,item,rating = data[i][0],data[i][6],data[i][9]
#     print(embedding(data[i][10],max_len).shape)
    X_user += [user_emb[user]]
    X_item += [item_emb[item]]
#     print(user_emb[user].shape,item_emb[item].shape)
    Y += [rating*1.0/5]
X_user = numpy.array(X_user)
X_item = numpy.array(X_item)
Y = numpy.array(Y)
# embd["and"].shape
X_user.shape
X_item.shape

(5000, 50, 50, 1)

In [139]:
model.compile(loss="mean_squared_error",optimizer=keras.optimizers.RMSprop(lr=0.002),metrics=["accuracy"])
model.fit([X_user,X_item],Y)

Epoch 1/10
5000/5000 [==============================] - 17s - loss: 0.0855 - acc: 0.3716    
Epoch 2/10
5000/5000 [==============================] - 15s - loss: 0.0701 - acc: 0.3904    
Epoch 3/10
5000/5000 [==============================] - 15s - loss: 0.0674 - acc: 0.3906    
Epoch 4/10
5000/5000 [==============================] - 15s - loss: 0.0667 - acc: 0.3912    
Epoch 5/10
5000/5000 [==============================] - 15s - loss: 0.0653 - acc: 0.3908    - ETA: 0s - loss: 0.0658 - acc: 
Epoch 6/10
5000/5000 [==============================] - 15s - loss: 0.0653 - acc: 0.3892    
Epoch 7/10
5000/5000 [==============================] - 15s - loss: 0.0640 - acc: 0.3890    
Epoch 8/10
5000/5000 [==============================] - 15s - loss: 0.0650 - acc: 0.3880    
Epoch 9/10
5000/5000 [==============================] - 15s - loss: 0.0637 - acc: 0.3884    
Epoch 10/10
5000/5000 [==============================] - 15s - loss: 0.0625 - acc: 0.3886    
